# Beam Search Example
This example demonstrates how to use the `BeamSearch` search algorithm with a simple reasoning task. The Beam Search algorithm is a heuristic search that explores a graph by expanding the most promising nodes in a limited set. Search is performed in a breadth-first manner, keeping track of the best candidates at each level. The algorithm continues until a stopping criterion is met, such as reaching a maximum number of steps or finding a satisfactory solution.

#### Installations
This example will use LMStudio for local model serving. Cot-Forge also supports major LLM providers like OpenAI, Cohere, and Anthropic. Download LMStudio [here](https://lmstudio.ai/), download a model and run a local server.

In [1]:
from cot_forge.llm import LMStudioProvider
from cot_forge.reasoning import BeamSearch, CoTBuilder
from cot_forge.reasoning.scorers import ProbabilityFinalAnswerScorer
from cot_forge.reasoning.verifiers import LLMJudgeVerifier

#### Problem definition
A [Caesar cipher](https://en.wikipedia.org/wiki/Caesar_cipher) is a type of substitution cipher in which each letter in the plaintext is replaced by a letter some fixed number of positions down or up the alphabet. We will attempt to use beam search to decode a simple Caesar cipher.

In [2]:
question = "Decipher this code: 73102109109112338811211510910134."
ground_truth = "Hello World!"

#### Using CoTBuilder class

In [3]:
# We'l use qwen2.5-14b-instruct-1m as the search_llm, scorer and verifier.
qwen2_5_14b = LMStudioProvider(model_name="qwen2.5-14b-instruct-1m")
# We'll use Llama 3.1 8b for our 'post_processing' LLM which converts our reasoning graphs into natural language.
llama3_1_8b = LMStudioProvider(model_name="meta-llama-3.1-8b-instruct")

In [ ]:
builder = CoTBuilder(
    search_llm=qwen2_5_14b,
    post_processing_llm=llama3_1_8b,
    search=BeamSearch(max_depth=3, beam_width=3, branching_factor=3),
    verifier=LLMJudgeVerifier(qwen2_5_14b, strict=False),
    scorer=ProbabilityFinalAnswerScorer(qwen2_5_14b)
)

In [9]:
search_result, reasoning = builder.process(
  question=question,
  ground_truth_answer=ground_truth,
  only_successful=False
)

Error decoding JSON: Invalid \escape: line 5 column 204 (char 909)
LLM generation and json parsing failed: Invalid response format. Expected a JSON string.. Retrying (1/5)...
Error decoding JSON: Invalid \escape: line 5 column 331 (char 1056)
LLM generation and json parsing failed: Invalid response format. Expected a JSON string.. Retrying (1/5)...


In [11]:
search_result

SearchResult(question=Decipher this code: 7310210910..., ground_truth_answer=Hello World!, num_terminal_nodes=2, success=False, successful_nodes=0, metadata={'depth': 2, 'reason': 'Max depth reached'})